Multi-task UnitedNet with PBMC dataset (peturbation of control cells with IF)

Both of the classification and the translation network's parameters are taken into account in the multi-task way orchestrated by UnitedNet.

In [1]:
# prepare dataset
# dataset from https://github.com/theislab/scgen-reproducibility, already preprocessed

import scanpy as sc

train_data = "data/pbmc/train_pbmc.h5ad"
valid_data = "data/pbmc/valid_pbmc.h5ad"

# pbmc data from scGen is already split into train and valid, but we will concatenate them to do a custom split
sc_data_train = sc.read_h5ad(train_data)
sc_data_valid = sc.read_h5ad(valid_data)
pbmc = sc_data_train.concatenate(sc_data_valid)

/g/kreshuk/katzalis/conda/miniforge3/envs/thesis/lib/python3.9/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/g/kreshuk/katzalis/conda/miniforge3/envs/thesis/lib/python3.9/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(
/g/kreshuk/katzalis/conda/miniforge3/envs/thesis/lib/python3.9/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/g/kreshuk/katzalis/conda/miniforge3/envs/thesis/lib/python3.9/site-packages/anndata/compat/__init__.py:311: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where ad

In [2]:
pbmc

AnnData object with n_obs × n_vars = 18868 × 6998
    obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type', 'batch'
    var: 'gene_symbol', 'n_cells'
    obsm: 'X_pca', 'X_tsne', 'X_umap'

In [3]:
pbmc.obs.groupby(['cell_type', 'condition']).size()

/tmp/ipykernel_3869093/191267336.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pbmc.obs.groupby(['cell_type', 'condition']).size()


cell_type    condition 
CD4T         control       2715
             stimulated    3483
CD14+Mono    control       2184
             stimulated     698
B            control        928
             stimulated    1105
CD8T         control        643
             stimulated     594
NK           control        571
             stimulated     733
FCGR3A+Mono  control       1232
             stimulated    2790
Dendritic    control        670
             stimulated     522
dtype: int64

In [4]:
pbmc.obs.groupby(['condition']).size()

/tmp/ipykernel_3869093/225387844.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pbmc.obs.groupby(['condition']).size()


condition
control       8943
stimulated    9925
dtype: int64

In [5]:
# simple inspection

# sc.pp.pca(pbmc)
# sc.pp.neighbors(pbmc)
# sc.tl.tsne(pbmc)
# sc.pl.tsne(pbmc, color=['condition', 'cell_type'], legend_loc='on data', legend_fontsize='small')

In [6]:
from scButterfly.split_datasets import unpaired_split_dataset_perturb


# create pairs of data using scButterfly technique
control_data = pbmc[pbmc.obs.condition == 'control']
stimulate_data = pbmc[pbmc.obs.condition == 'stimulated']

control_data.obs.index = [str(i) for i in range(control_data.X.shape[0])]
stimulate_data.obs.index = [str(i) for i in range(stimulate_data.X.shape[0])]

cell_type_list = list(control_data.obs.cell_type.cat.categories)

In [7]:
id_list, id_list_dict = unpaired_split_dataset_perturb(control_data, stimulate_data)

optimal transport array torch.Size([2715, 3483])
CD4T, control num 2715 stimulate num 3483

optimal transport array torch.Size([2184, 698])
CD14+Mono, control num 2184 stimulate num 698

optimal transport array torch.Size([928, 1105])
B, control num 928 stimulate num 1105

optimal transport array torch.Size([643, 594])
CD8T, control num 643 stimulate num 594

optimal transport array torch.Size([571, 733])
NK, control num 571 stimulate num 733

optimal transport array torch.Size([1232, 2790])
FCGR3A+Mono, control num 1232 stimulate num 2790

optimal transport array torch.Size([670, 522])
Dendritic, control num 670 stimulate num 522


Start CD4T
Batch list ['Dendritic', 'B', 'FCGR3A+Mono', 'CD14+Mono', 'CD4T', 'NK', 'CD8T']
Test batch control ['Dendritic']
Validation batch control ['B', 'FCGR3A+Mono', 'CD14+Mono', 'CD4T', 'NK', 'CD8T']
Train batch control ['B', 'FCGR3A+Mono', 'CD14+Mono', 'CD4T', 'NK', 'CD8T']
Test batch stimulated ['Dendritic']
Validation batch stimulated ['B', 'FCGR3A+

In [8]:
#print(len(id_list[0]))
for idx, cell_type in enumerate(cell_type_list):
    print(cell_type)
    train_id_control, train_id_peturb, validation_id_control, validation_id_peturb, test_id_control, test_id_peturb = id_list[idx]

    control = len(train_id_control) + len(validation_id_control) + len(test_id_control)
    peturb = len(train_id_peturb) + len(validation_id_peturb) + len(test_id_peturb) 
    control_dict = id_list_dict[cell_type]["control"]
    peturb_dict = id_list_dict[cell_type]["stimulated"]
    print("control", len(train_id_control), len(validation_id_control), len(test_id_control), control)
    print("control", len(control_dict["train"]), len(control_dict["validation"]), len(control_dict["test"]))
    print("peturb", len(train_id_peturb), len(validation_id_peturb), len(test_id_peturb), peturb)    
    print("peturb", len(peturb_dict["train"]), len(peturb_dict["validation"]), len(peturb_dict["test"]))
    print()
    
# Some notes due to the lack of code readability from scButterfly:
# - the rows of the id_list correspond to the cell types, but they don't have any meaning.
# - the rows actually correspond to k-fold cross validation. One batch is a cell type that is held out.
# - the cell_type of the row and the one that is held out are irrelevant.
# - for the test, the control and the stimulated can have different size. But for train and validation we need pairs.

CD4T
control 6616 1657 670 8943
control 6616 1657 670
peturb 6616 1657 522 8795
peturb 6616 1657 522

CD14+Mono
control 6410 1605 928 8943
control 6410 1605 928
peturb 6410 1605 1105 9120
peturb 6410 1605 1105

B
control 6167 1544 1232 8943
control 6167 1544 1232
peturb 6167 1544 2790 10501
peturb 6167 1544 2790

CD8T
control 5405 1354 2184 8943
control 5405 1354 2184
peturb 5405 1354 698 7457
peturb 5405 1354 698

NK
control 4980 1248 2715 8943
control 4980 1248 2715
peturb 4980 1248 3483 9711
peturb 4980 1248 3483

FCGR3A+Mono
control 6696 1676 571 8943
control 6696 1676 571
peturb 6696 1676 733 9105
peturb 6696 1676 733

Dendritic
control 6638 1662 643 8943
control 6638 1662 643
peturb 6638 1662 594 8894
peturb 6638 1662 594



In [9]:
# create adatas for pair of control and stimulated
import numpy as np


batch = cell_type_list[0]
id_list_batch = id_list_dict[batch]
control = id_list_batch["control"]
peturb = id_list_batch["stimulated"]
train_id_control = control["train"]
validation_id_control = control["validation"]
test_id_control = control["test"]
train_id_peturb = peturb["train"]
validation_id_peturb = peturb["validation"]
test_id_peturb = peturb["test"]

control_train = pbmc[train_id_control]
control_valid = pbmc[validation_id_control]
control_test = pbmc[test_id_control]

peturb_train = pbmc[train_id_peturb]
peturb_valid = pbmc[validation_id_peturb]
peturb_test = pbmc[test_id_peturb]

all = [control_train, control_valid, control_test, peturb_train, peturb_valid, peturb_test]
for adata in all:
    adata.obs["label"] = list(adata.obs["cell_type"])
    adata.X = adata.X.toarray()
    print("min", np.max(adata.X))
    print("max", np.min(adata.X))
    print()

adatas_train = [control_train, peturb_train]
adatas_valid = [control_valid, peturb_valid]
adatas_test = [control_test, peturb_test]

features_num = adatas_train[0].n_vars
adatas_train


min 7.0346947
max 0.0

min 6.822089
max 0.0

min 6.7637987
max 0.0

min 6.879698
max 0.0

min 7.0346947
max 0.0

min 5.961386
max 0.0



[AnnData object with n_obs × n_vars = 6616 × 6998
     obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type', 'batch', 'label'
     var: 'gene_symbol', 'n_cells'
     obsm: 'X_pca', 'X_tsne', 'X_umap',
 AnnData object with n_obs × n_vars = 6616 × 6998
     obs: 'condition', 'n_counts', 'n_genes', 'mt_frac', 'cell_type', 'batch', 'label'
     var: 'gene_symbol', 'n_cells'
     obsm: 'X_pca', 'X_tsne', 'X_umap']

In [13]:
from unitednet.interface import UnitedNet


pbmc_config = {
    "train_batch_size": 512,
    "finetune_batch_size": 5000,
    "transfer_batch_size": 512,
    "train_epochs": 10,
    "finetune_epochs": 10,
    "transfer_epochs": 20,
    "train_task": "supervised_group_identification", # -> translation, classification
    "finetune_task": None,
    "transfer_task": None,
    "train_loss_weight": None,
    "finetune_loss_weight": None,
    "transfer_loss_weight": None,
    "lr": 0.1,
    "checkpoint": 1,
    "n_head": 1,
    "noise_level":[0,0],
    "fuser_type":"WeightedMean",
    "encoders": [
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
    ],
    "latent_projector": None,
    "decoders": [
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "sigmoid"],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", None],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
    ],
    "discriminators": [
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
    ],
    "projectors": {
        "input": 64,
        "hiddens": [],
        "output": 100,
        "use_biases": [True],
        "dropouts": [0],
        "activations": ["relu"],
        "use_batch_norms": [False],
        "use_layer_norms": [True],
    },
    "clusters": {
        "input": 100,
        "hiddens": [],
        "output": len(cell_type_list),
        "use_biases": [False],
        "dropouts": [0],
        "activations": [None],
        "use_batch_norms": [False],
        "use_layer_norms": [False],
    },
}

device="cuda:0"
test_batch= batch
root_save_path = f"saved_results/thesis/pbmc/multi_task"
import torch
print(torch.cuda.is_available())

model = UnitedNet(f"{root_save_path}/{test_batch}", device=device, technique=pbmc_config)

True


In [14]:
model.train(adatas_train, verbose=True)
#model.finetune(adatas_train, verbose=True)
#model.transfer(adatas_train, adatas_transfer = adatas_test, verbose=True)
#print(model.evaluate(adatas_test))
# evaluation should be different for peturbation

training


  0%|          | 0/10 [00:00<?, ?it/s]



Losses                         Value
-------------------------  ---------
cross_entropy_loss_head_0  0.0394749
best_head                  0
best model saved at saved_results/thesis/pbmc/multi_task/CD4T/CD4T/train_best.pt 

model saved at saved_results/thesis/pbmc/multi_task/CD4T/CD4T/train_epoch_1.pt 



 10%|█         | 1/10 [00:04<00:43,  4.82s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05389484 -0.06721308]
                                [-4.47238731 -4.47238731]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  451    0    0    0    1    1]
                                [   3    0 2329   15    1    1    0]
                                [   1    0    1  386    0    0    2]
                                [   0    0    0    0  339    0    0]
                                [   2   73    0    0    3 1781    0]
                                [   0    0    0    1    0    0  480]]
Accuracy                       0.9841293833131801
Adjusted Rand Index            0.9689359985668443
Normalized Mutual Information  0.9585519299091642


Losses                         Value
-------------------------  ---------
cross_entropy_loss_head_0  0.0060315
best_head                  

...



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05599609 -0.05599609]
                                [-0.00264814 -0.00264814]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2348    1    0    0    0]
                                [   0    0    1  389    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0    0    0    0    0 1859    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       0.9996977025392987
Adjusted Rand Index            0.9993014689750885
Normalized Mutual Information  0.9985711329368417


Losses                           Value
-------------------------  -----------
cross_entropy_loss_head_0  0.000221031
best_head            

 90%|█████████ | 9/10 [00:46<00:05,  5.61s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05599609 -0.05599609]
                                [-0.00237198 -0.00237198]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0    0    0    0    0 1859    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       1.0
Adjusted Rand Index            1.0
Normalized Mutual Information  1.0
model saved at saved_results/thesis/pbmc/multi_task/CD4T/CD4T/train_epoch_10.pt 



100%|██████████| 10/10 [00:51<00:00,  5.13s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05599609 -0.05599609]
                                [-0.00229128 -0.00229128]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0    0    1    0    1 1857    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       0.9996977025392987
Adjusted Rand Index            0.9991828632765513
Normalized Mutual Information  0.9985171210790332


Single-task translation 

The classification network's parameters are ignored.

In [16]:
from unitednet.interface import UnitedNet


pbmc_config_translation = {
    "train_batch_size": 512,
    "finetune_batch_size": 5000,
    "transfer_batch_size": 512,
    "train_epochs": 10,
    "finetune_epochs": 10,
    "transfer_epochs": 20,
    "train_task": "cross_model_prediction", # translation
    "finetune_task": None,
    "transfer_task": None,
    "train_loss_weight": None,
    "finetune_loss_weight": None,
    "transfer_loss_weight": None,
    "lr": 0.1,
    "checkpoint": 1,
    "n_head": 1,
    "noise_level":[0,0],
    "fuser_type":"WeightedMean",
    "encoders": [
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
    ],
    "latent_projector": None,
    "decoders": [
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "sigmoid"],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", None],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
    ],
    "discriminators": [
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
    ],
    "projectors": {
        "input": 64,
        "hiddens": [],
        "output": 100,
        "use_biases": [True],
        "dropouts": [0],
        "activations": ["relu"],
        "use_batch_norms": [False],
        "use_layer_norms": [True],
    },
    "clusters": {
        "input": 100,
        "hiddens": [],
        "output": len(cell_type_list),
        "use_biases": [False],
        "dropouts": [0],
        "activations": [None],
        "use_batch_norms": [False],
        "use_layer_norms": [False],
    },
}

device="cuda:0"
test_batch= batch
root_save_path = f"saved_results/thesis/pbmc/translation"

model_translation = UnitedNet(f"{root_save_path}/{test_batch}", device=device, technique=pbmc_config_translation)

In [17]:
model_translation.train(adatas_train, verbose=True)


training


  0%|          | 0/10 [00:00<?, ?it/s]



Losses                    Value
-------------------  ----------
discriminator_loss   0.0595083
reconstruction_loss  0.0411343
generator_loss       0.00795011
contrastive_loss     9.3067
translation_loss     0.0411343
best_head            0
best model saved at saved_results/thesis/pbmc/translation/CD4T/CD4T/train_best.pt 

model saved at saved_results/thesis/pbmc/translation/CD4T/CD4T/train_epoch_1.pt 



 10%|█         | 1/10 [00:04<00:43,  4.88s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05620258 -0.05620258]
                                [-4.02247524 -4.02247524]]
Confusion Matrix               [[   7    0  626    0    2   61   49]
                                [   0    0  113    0    0  338    2]
                                [  15    0 1863    1   13  245  212]
                                [   3    0  297    1    1   61   27]
                                [   0    0  188    0    0  128   23]
                                [   0    0  565    0    1 1260   33]
                                [   2    0  365    0    1   67   46]]
Accuracy                       0.4801995163240629
Adjusted Rand Index            0.13020679729601353
Normalized Mutual Information  0.1497576428276985


Losses                   Value
-------------------  ---------
discriminator_loss   0.0555364
reconstruction_loss  0.0382109
generator_los

...



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05606649 -0.05606649]
                                [-0.00237421 -0.00237421]]
Confusion Matrix               [[  13    0  274    1    0  457    0]
                                [   0    0  125    0    0  328    0]
                                [  20    1  937    0    0 1391    0]
                                [   2    0   99    0    0  289    0]
                                [   5    0  139    0    0  195    0]
                                [  10    1  637    0    0 1211    0]
                                [   5    0  106    0    0  370    0]]
Accuracy                       0.32663240628778717
Adjusted Rand Index            -0.006246173097579558
Normalized Mutual Information  0.00868536358440616
model saved at saved_results/thesis/pbmc/translation/CD4T/CD4T/train_epoch_9.pt 



 90%|█████████ | 9/10 [00:32<00:03,  3.42s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05606646 -0.05606649]
                                [-0.00198999 -0.00198999]]
Confusion Matrix               [[   4    1  282    0    0  458    0]
                                [   0    0  129    0    0  324    0]
                                [  11    1  956    0    0 1377    4]
                                [   1    0  101    0    0  288    0]
                                [   0    0  150    0    0  189    0]
                                [   7    1  663    0    0 1188    0]
                                [   1    0  108    0    0  371    1]]
Accuracy                       0.3248186215235792
Adjusted Rand Index            -0.006870036655446086
Normalized Mutual Information  0.008766087338492115


Losses                   Value
-------------------  ---------
discriminator_loss   0.0555608
reconstruction_loss  0.0369381
generator

100%|██████████| 10/10 [00:36<00:00,  3.61s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[-0.05582297 -0.05582298]
                                [-0.00215461 -0.00215461]]
Confusion Matrix               [[   5    0  275    0    0  463    2]
                                [   2    0  123    0    0  328    0]
                                [  13    0  946    0    0 1384    6]
                                [   1    0  101    0    0  288    0]
                                [   3    0  144    0    1  190    1]
                                [   7    0  635    0    1 1214    2]
                                [   2    0  106    0    0  371    2]]
Accuracy                       0.32769044740024184
Adjusted Rand Index            -0.006267101988215142
Normalized Mutual Information  0.008452413107276518


Single-task cell type annotation

The translation networks' parameters are ignored.

In [18]:
from unitednet.interface import UnitedNet


pbmc_config_classification = {
    "train_batch_size": 512,
    "finetune_batch_size": 5000,
    "transfer_batch_size": 512,
    "train_epochs": 10,
    "finetune_epochs": 10,
    "transfer_epochs": 20,
    "train_task": "supervised_group_identification_only", # classification
    "finetune_task": None,
    "transfer_task": None,
    "train_loss_weight": None,
    "finetune_loss_weight": None,
    "transfer_loss_weight": None,
    "lr": 0.1,
    "checkpoint": 1,
    "n_head": 1,
    "noise_level":[0,0],
    "fuser_type":"WeightedMean",
    "encoders": [
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
        {
            "input": features_num,
            "hiddens": [64, 64],
            "output": 64,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "relu"],
            "use_batch_norms": [True, True, True],
            "use_layer_norms": [False, False, False],
            "is_binary_input": False,
        },
    ],
    "latent_projector": None,
    "decoders": [
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", "sigmoid"],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
        {
            "input": 64,
            "hiddens": [64, 64],
            "output": features_num,
            "use_biases": [True, True, True],
            "dropouts": [0, 0, 0],
            "activations": ["relu", "relu", None],
            "use_batch_norms": [False, False, False],
            "use_layer_norms": [False, False, False],
        },
    ],
    "discriminators": [
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
        {
            "input": features_num,
            "hiddens": [64],
            "output": 1,
            "use_biases": [True, True],
            "dropouts": [0, 0],
            "activations": ["relu", "sigmoid"],
            "use_batch_norms": [False, False],
            "use_layer_norms": [False, True],
        },
    ],
    "projectors": {
        "input": 64,
        "hiddens": [],
        "output": 100,
        "use_biases": [True],
        "dropouts": [0],
        "activations": ["relu"],
        "use_batch_norms": [False],
        "use_layer_norms": [True],
    },
    "clusters": {
        "input": 100,
        "hiddens": [],
        "output": len(cell_type_list),
        "use_biases": [False],
        "dropouts": [0],
        "activations": [None],
        "use_batch_norms": [False],
        "use_layer_norms": [False],
    },
}

device="cuda:0"
test_batch= batch
root_save_path = f"saved_results/thesis/pbmc/classification"

model_classification = UnitedNet(f"{root_save_path}/{test_batch}", device=device, technique=pbmc_config_classification)

In [19]:
model_classification.train(adatas_train, verbose=True)


training


  0%|          | 0/10 [00:00<?, ?it/s]



Losses                        Value
-------------------------  --------
cross_entropy_loss_head_0  0.226991
best_head                  0
best model saved at saved_results/thesis/pbmc/classification/CD4T/CD4T/train_best.pt 

model saved at saved_results/thesis/pbmc/classification/CD4T/CD4T/train_epoch_1.pt 



 10%|█         | 1/10 [00:04<00:37,  4.22s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[ -452.13931274  -348.13018799]
                                [-4292.48925781  -453.43054199]]
Confusion Matrix               [[ 740    0    2    1    2    0    0]
                                [   0  452    0    0    0    1    0]
                                [   6    7 2200  135    0    1    0]
                                [   0    0    0  381    0    0    9]
                                [   0    2    0    0  334    3    0]
                                [   3  860    2    0   64  923    7]
                                [   0    0    1    5    0    0  475]]
Accuracy                       0.8320737605804112
Adjusted Rand Index            0.7605106205207351
Normalized Mutual Information  0.8246357413941272


Losses                         Value
-------------------------  ---------
cross_entropy_loss_head_0  0.0244222
best_head      

...



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[ -447.47564697  -344.02441406]
                                [-3830.68359375  -332.47918701]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0   18    0    0    0 1841    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       0.997279322853688
Adjusted Rand Index            0.9947218098479961
Normalized Mutual Information  0.9919325919933397
model saved at saved_results/thesis/pbmc/classification/CD4T/CD4T/train_epoch_9.pt 



 90%|█████████ | 9/10 [00:38<00:04,  4.35s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[ -449.81253052  -346.45788574]
                                [-4052.74536133  -403.01547241]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0   50    0    0    0 1809    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       0.9924425634824667
Adjusted Rand Index            0.9855006098757555
Normalized Mutual Information  0.9822068752851634


Losses                           Value
-------------------------  -----------
cross_entropy_loss_head_0  0.000172609
best_head

100%|██████████| 10/10 [00:42<00:00,  4.24s/it]



Metrics                        Value
-----------------------------  --------------------------------------
R2                             [[ -451.26821899  -345.86209106]
                                [-3976.87792969  -398.92633057]]
Confusion Matrix               [[ 745    0    0    0    0    0    0]
                                [   0  453    0    0    0    0    0]
                                [   0    0 2349    0    0    0    0]
                                [   0    0    0  390    0    0    0]
                                [   0    0    0    0  339    0    0]
                                [   0    0    0    0    0 1859    0]
                                [   0    0    0    0    0    0  481]]
Accuracy                       1.0
Adjusted Rand Index            1.0
Normalized Mutual Information  1.0
